In [1]:
import netgen.gui
from netgen.geom2d import unit_square
from ngsolve import *

### Initializations

In [20]:
#epsilon = 0.01
# h = uniformly refined mesh
# order = k

In [23]:
## Parameter setup
orders = [1, 2, 3]
beta = (2,1)
mesh_size = [1.0, 0.5, 0.25, 0.1250, 0.0625, 0.0313]
eps = 0.01

# Exact Solution
p = lambda x: x + (exp(beta[0]*x/eps)-1)/(1-exp(beta[0]/eps))
q = lambda y: y + (exp(beta[1]*y/eps)-1)/(1-exp(beta[1]/eps))
exact = p(x) * q(y)

coeff = beta[1]*p(x) + beta[0]*q(y)

alpha = 100

### Exact Solution for Convection-Diffusion Equation

In [21]:
orders = 1
mesh_size = 0.05
mesh = Mesh(unit_square.GenerateMesh(maxh=mesh_size))
fes = L2(mesh, order=orders, dirichlet="bottom|right|left|top")
gfu = GridFunction(fes)
Draw(exact, mesh, 'exact')

### Discontinuous Galerkin for Convection-Diffusion with upwind

In [24]:
for order in orders:
    for size in mesh_size:
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        fes = L2(mesh, order=order, dgjumps=True)
        u, v = fes.TnT()


        jump_u = u-u.Other()
        jump_v = v-v.Other()
        n = specialcf.normal(2)
        mean_dudn = eps * 0.5*n * (grad(u) + grad(u.Other()))
        mean_dvdn = eps * 0.5*n * (grad(v) + grad(v.Other()))



        h = specialcf.mesh_size
        a = BilinearForm(fes)

        diffusion = eps * grad(u) * grad(v) * dx \
            +alpha * eps * order ** 2/ h * jump_u * jump_v * dx(skeleton=True) \
            +(-mean_dudn * jump_v-mean_dvdn * jump_u) * dx(skeleton=True) \
            +alpha * eps * order ** 2/h * u * v * ds(skeleton=True) \
            + eps * (-n * grad(u) * v -n * grad(v) * u) * ds(skeleton=True)

        a += diffusion
        a.Assemble()

 
        f = LinearForm(fes)
        f += SymbolicLFI(coeff * v )
        f.Assemble()


        gfu = GridFunction(fes, name="uDG")
        gfu.vec.data = a.mat.Inverse() * f.vec
        Draw (gfu)
        
        b = CoefficientFunction((beta[0],beta[1]) )

        uup = IfPos(b*n, u, u.Other())

        convection = -b * u * grad(v) * dx + b * n * uup * jump_v * dx(skeleton=True)+ IfPos(b*n,b*n,0)*u*v*ds(skeleton=True)

        acd = BilinearForm(fes)
        acd += diffusion + convection

        acd.Assemble()

        gfu = GridFunction(fes)
        gfu.vec.data = acd.mat.Inverse(freedofs=fes.FreeDofs(),inverse="umfpack") * f.vec
        
        print ('Order:', order, 'Mesh Size:', size , "L2-error:", sqrt (Integrate ((gfu-exact)*(gfu-exact), mesh)))
        #Draw(exact,mesh,"u")
        #Draw(gfu,mesh,"uh")
    print('......................................................................')
    
    

Order: 1 Mesh Size: 1.0 L2-error: 0.18858271771985755
Order: 1 Mesh Size: 0.5 L2-error: 0.15900789149842628
Order: 1 Mesh Size: 0.25 L2-error: 0.13979275416656903
Order: 1 Mesh Size: 0.125 L2-error: 0.12036257425598847
Order: 1 Mesh Size: 0.0625 L2-error: 0.07566263907754531
Order: 1 Mesh Size: 0.0313 L2-error: 0.03458493405768635
......................................................................
Order: 2 Mesh Size: 1.0 L2-error: 0.5012289412744237
Order: 2 Mesh Size: 0.5 L2-error: 0.2385774927812544
Order: 2 Mesh Size: 0.25 L2-error: 0.14208099659653617
Order: 2 Mesh Size: 0.125 L2-error: 0.07413411487242062
Order: 2 Mesh Size: 0.0625 L2-error: 0.03068731046845547
Order: 2 Mesh Size: 0.0313 L2-error: 0.009577962690863473
......................................................................
Order: 3 Mesh Size: 1.0 L2-error: 0.37269916618461196
Order: 3 Mesh Size: 0.5 L2-error: 0.22402708987183748
Order: 3 Mesh Size: 0.25 L2-error: 0.10116813716439363
Order: 3 Mesh Size: 0.125 L2-e

### Hybrid Discontinuous Galerkin Convection-Diffusion with upwind

In [25]:
for order in orders:
    for size in mesh_size:
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order)
        F = FacetFESpace(mesh, order=order, dirichlet="bottom|left|right|top")

        fes = FESpace([V,F])
        u, uhat = fes.TrialFunction()
        v,vhat = fes.TestFunction()

        jump_u = u-uhat
        jump_v = v-vhat

        condense = True
        h = specialcf.mesh_size
        n = specialcf.normal(mesh.dim)

        a = BilinearForm(fes, condense=condense)
        dS = dx(element_boundary=True)

        diffusion = eps * grad(u) * grad(v) * dx + \
            alpha * eps * order ** 2/h * jump_u * jump_v * dS + \
            eps * (-grad(u) * n * jump_v - grad(v) * n * jump_u) * dS

                     
        b = CoefficientFunction((beta[0],beta[1])) 

        uup = IfPos(b * n, u, uhat)
              
        convection = -b * u * grad(v) * dx + b * n * uup * jump_v * dS

        a += convection + diffusion
        
        a.Assemble()
                
        f = LinearForm(fes)
        f += SymbolicLFI(coeff * v)
        f.Assemble()

        gfu = GridFunction(fes)
        
        if not condense:
            inv = a.mat.Inverse(fes.FreeDofs(), "umfpack")
            gfu.vec.data = inv * f.vec
        else:
            f.vec.data += a.harmonic_extension_trans * f.vec

            inv = a.mat.Inverse(fes.FreeDofs(True), "umfpack")
            gfu.vec.data = inv * f.vec

            gfu.vec.data += a.harmonic_extension * gfu.vec
            gfu.vec.data += a.inner_solve * f.vec
            
        print('Order:', order, 'Mesh Size:', size , "L2-error:", sqrt (Integrate ((gfu.components[0] - exact)*(gfu.components[0]- exact), mesh)))
                            
    print('......................................................................')

Order: 1 Mesh Size: 1.0 L2-error: 0.19071450801268824
Order: 1 Mesh Size: 0.5 L2-error: 0.1560975788582526
Order: 1 Mesh Size: 0.25 L2-error: 0.12523467925975507
Order: 1 Mesh Size: 0.125 L2-error: 0.10361254552408605
Order: 1 Mesh Size: 0.0625 L2-error: 0.06835496957049685
Order: 1 Mesh Size: 0.0313 L2-error: 0.03275266369323318
......................................................................
Order: 2 Mesh Size: 1.0 L2-error: 0.47926937524537844
Order: 2 Mesh Size: 0.5 L2-error: 0.1844554339990846
Order: 2 Mesh Size: 0.25 L2-error: 0.12559045299673152
Order: 2 Mesh Size: 0.125 L2-error: 0.07046735288047248
Order: 2 Mesh Size: 0.0625 L2-error: 0.02993459876813486
Order: 2 Mesh Size: 0.0313 L2-error: 0.009412773713045755
......................................................................
Order: 3 Mesh Size: 1.0 L2-error: 0.37233642334277706
Order: 3 Mesh Size: 0.5 L2-error: 0.19922746993006343
Order: 3 Mesh Size: 0.25 L2-error: 0.09830238800473401
Order: 3 Mesh Size: 0.125 L2-e

### Solving Mass Matrix problem

In [26]:
orders = [1, 2, 3, 4]
mesh_size = [1.0, 0.5, 0.25, 0.1250, 0.0625, 0.0313]
for order in orders:
    for size in mesh_size:
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        fes = L2(mesh, order=order, dirichlet="bottom|right|left|top")
        u, v = fes.TnT()
        
        b = CoefficientFunction((beta[0],beta[1]))        
        
        a = BilinearForm(fes, symmetric=False)
        a += SymbolicBFI((eps *  grad(u) * grad(v)) + (b * grad(u) * v))
        a.Assemble()
        
        m = BilinearForm(fes, symmetric=False)
        m += u * v * dx
        m.Assemble()
        
        f = LinearForm(fes)
        f += SymbolicLFI(coeff * v)
        f.Assemble()
        
        mstar = m.mat.CreateMatrix()
        mstar.AsVector().data = m.mat.AsVector() + a.mat.AsVector()
        invmstar = mstar.Inverse(freedofs=fes.FreeDofs())
        
                
        gfu = GridFunction(fes)
        res = gfu.vec.CreateVector()
        
        res.data = f.vec  - a.mat * gfu.vec
        gfu.vec.data = invmstar * res
        #Draw(gfu, mesh, 'test')

        print('Order:', order, 'Mesh Size:', size , "L2 Error:", sqrt (Integrate ((gfu - exact)*(gfu- exact), mesh)))
            
    print('......................................................................')

Order: 1 Mesh Size: 1.0 L2 Error: 0.8551041032257676
Order: 1 Mesh Size: 0.5 L2 Error: 2.930126258157547
Order: 1 Mesh Size: 0.25 L2 Error: 1.7976126329027853
Order: 1 Mesh Size: 0.125 L2 Error: 1.3843012728860409
Order: 1 Mesh Size: 0.0625 L2 Error: 1.322609837525558
Order: 1 Mesh Size: 0.0313 L2 Error: 1.3131797044528166
......................................................................
Order: 2 Mesh Size: 1.0 L2 Error: 54.22381397395486
Order: 2 Mesh Size: 0.5 L2 Error: 3.815941049147384
Order: 2 Mesh Size: 0.25 L2 Error: 1.5368200780323733
Order: 2 Mesh Size: 0.125 L2 Error: 1.5992470376608572
Order: 2 Mesh Size: 0.0625 L2 Error: 1.7650217421709522
Order: 2 Mesh Size: 0.0313 L2 Error: 1.3069457957628945
......................................................................
Order: 3 Mesh Size: 1.0 L2 Error: 1.413889367120495
Order: 3 Mesh Size: 0.5 L2 Error: 1.450927263830939
Order: 3 Mesh Size: 0.25 L2 Error: 1.306849919496648
Order: 3 Mesh Size: 0.125 L2 Error: 1.2873748738308

### Enrichment

#### Wall function Enrichment with Mass Matrix

In [20]:
orders = [1, 2, 3, 4]
#orders = [4]
mesh_size = [1.0, 0.5, 0.25, 0.1250, 0.0625, 0.0313]
# mesh_size = [0.25]
for order in orders:
    for size in mesh_size:
        mesh = Mesh(unit_square.GenerateMesh(maxh=size))
        V = L2(mesh, order=order, dirichlet="bottom|right|left|top")

        Q = L2(mesh, order=0)
        ba = BitArray(mesh.ne)
        ba.Set()
        ba[1] = False
        ba[0] = False
        
        Q = Compress(Q, active_dofs=ba)

        fes = FESpace([V, Q])

        #gfu = GridFunction(fes)

        (u, p), (v, q) = fes.TnT()

        p = coeff * p
        q = coeff * q

        grad_u = grad(u) + CoefficientFunction((coeff.Diff(x), coeff.Diff(y))) * p
        grad_v = grad(v) + CoefficientFunction((coeff.Diff(x), coeff.Diff(y))) * q
        
        u = u + p
        v = v + q

        b = CoefficientFunction((beta[0],beta[1]))        

        a = BilinearForm(fes, symmetric=False)
        a += SymbolicBFI((eps *  grad_u * grad_v) + (b * grad_u * v))
        a.Assemble()

        m = BilinearForm(fes, symmetric=False)
        m += u * v * dx
        m.Assemble()

        f = LinearForm(fes)
        f += SymbolicLFI(coeff * v)
        f.Assemble()

        mstar = m.mat.CreateMatrix()
        mstar.AsVector().data = m.mat.AsVector() + a.mat.AsVector()
        invmstar = mstar.Inverse(freedofs=fes.FreeDofs())

        gfu = GridFunction(fes)
        res = gfu.vec.CreateVector()

        res.data = f.vec  - a.mat * gfu.vec
        gfu.vec.data = invmstar * res
        u2 = gfu.components[0] + gfu.components[1] * coeff
        #Draw(u2, mesh, 'test')

        print('Order:', order, ' Mesh Size:', size , "L2 Error:", sqrt (Integrate ((u2 - exact)*(u2- exact), mesh)))
    print('...........................................................')

Order: 1  Mesh Size: 1.0 L2 Error: 0.8551041032257672
Order: 1  Mesh Size: 0.5 L2 Error: 3.2699571696735714
Order: 1  Mesh Size: 0.25 L2 Error: 3.724517695190019
Order: 1  Mesh Size: 0.125 L2 Error: 1.5005144839181277
Order: 1  Mesh Size: 0.0625 L2 Error: 7.415612914875263
Order: 1  Mesh Size: 0.0313 L2 Error: 1.4195764128277457
...........................................................
Order: 2  Mesh Size: 1.0 L2 Error: 54.22381397395486
Order: 2  Mesh Size: 0.5 L2 Error: 1.9740121624938607
Order: 2  Mesh Size: 0.25 L2 Error: 1.3871067300370616
Order: 2  Mesh Size: 0.125 L2 Error: 1.3025167605082633
Order: 2  Mesh Size: 0.0625 L2 Error: 1.2975596255620612
Order: 2  Mesh Size: 0.0313 L2 Error: 6.659791211953927
...........................................................
Order: 3  Mesh Size: 1.0 L2 Error: 1.413889367120495
Order: 3  Mesh Size: 0.5 L2 Error: 4.084001614307023
Order: 3  Mesh Size: 0.25 L2 Error: 1.3017827208790902
Order: 3  Mesh Size: 0.125 L2 Error: 1.572908745206595
Or

#### Wall function Enrichment with DG

In [6]:
size = 0.25
order = 4
mesh = Mesh(unit_square.GenerateMesh(maxh=size))
V = L2(mesh, order=order, dirichlet="bottom|right|left|top")

Q = L2(mesh, order=0)
ba = BitArray(mesh.ne)
ba.Set()
ba[0] = False
        
Q = Compress(Q, active_dofs=ba)

fes = FESpace([V, Q])

gfu = GridFunction(fes)
        

(u, p), (v, q) = fes.TnT()

p = coeff * p
q = coeff * q

grad_u = grad(u) + CoefficientFunction((coeff.Diff(x), coeff.Diff(y))) * p
grad_v = grad(v) + CoefficientFunction((coeff.Diff(x), coeff.Diff(y))) * q

grad_u_Other = grad(u.Other()) + CoefficientFunction((coeff.Diff(x), coeff.Diff(y))) * p
grad_v_Other = grad(v.Other()) + CoefficientFunction((coeff.Diff(x), coeff.Diff(y))) * q

b = CoefficientFunction((beta[0],beta[1]))  

jump_u = u - u.Other()
jump_v = v - v.Other()
n = specialcf.normal(2)

mean_dudn = eps * 0.5 * n * (grad_u + grad_u_Other)
mean_dvdn = eps * 0.5 * n * (grad_v + grad_v_Other)

In [7]:
h = specialcf.mesh_size
a = BilinearForm(fes)

diffusion = grad_u*grad_v * dx \
    +alpha*order**2/h*jump_u*jump_v * dx(skeleton=True) \
    +(-mean_dudn*jump_v-mean_dvdn*jump_u) * dx(skeleton=True) \
    +alpha*order**2/h*u*v * ds(skeleton=True) \
    +(-n*grad_u*v-n*grad_v*u)* ds(skeleton=True)

a += diffusion
a.Assemble()


NgException: SparseMatrixTM::AddElementMatrix: illegal dnumsin Assemble BilinearForm 'biform_from_py'


#### Wall function Enrichment with HDG